In [268]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## Choropleth Map
The goal of this homework is to build a [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) showing how much research funding goes to each Swiss canton. We use the [P3 database](http://p3.snf.ch/) from the Swiss National Science Foundation. Since the data only contain the names of universities, we need to map each university to a location – and more specifically a canton. To this end, we use the Google Places and Google Geocoding APIs. Finally the map is created in Javascript using the [Leaflet](http://leafletjs.com/) library.

### Importing the data
We get the data from the CSV file. In German, the text *Nicht zuteilbar - NA* means that there are no data so we specify we want the NaN value instead. We also specify that the index should be the *Project number* column. As seen below this index is unique.

In [ ]:
df = pd.read_csv('P3_GrantExport.csv', sep=';', index_col=0, na_values=['Nicht zuteilbar - NA'])

In [ ]:
df.index.is_unique

### Locations of universities
As seen below, the names of universities often contains a dash followed by the short name of the institute. To get more accurate results when using the Google API, we want to be able to remove this part of the name. The `trim_university_name(full_name)` function extracts only the first part of the university name.

We also define a `search_uni(university_name)` function which returns a tuple containing the canton the university is in, the latitude and the longitude. The canton name is represented with two letters (e.g. GE for Geneva). All of these functions are defined in the utils.py file.

In [ ]:
from utils import trim_university_name, search_uni

For instance, we can get the location of EPFL. Please keep in mind that you need a Google API key to use the `search_uni()` function. The key has to be in a JSON file named `google_api_key.json`, associated with the `api_key` key.

In [ ]:
search_uni(trim_university_name('Ecole Polytechnique Fédérale de Lausanne - EPFL'))

We want to create a CSV file containing the list of all universities that receive funding with their associated location and canton. First we create a series containing the names of all universities.

In [ ]:
uni_names = df.University.drop_duplicates().dropna()
uni_names.head()

Our dataset contains 76 universities

In [ ]:
uni_names.count()

Then, we want to associate each university to its location. We create a dictionary containing this mapping that we use to create a Pandas dataframe. **Executing this cell sends a lot of requests to the Google API, you may prefer to skip it and load the CSV in the next cell!**

In [ ]:
#uni_infos = {uni: search_uni(trim_university_name(uni)) for uni in uni_names}
#uni_data = uni_names.map(lambda s: uni_infos[s] if uni_infos[s] is not None else (np.nan,) * 3)
#unis = pd.DataFrame(uni_data.tolist(), columns=['canton', 'latitude', 'longitude'], index=uni_names)
#unis.index.name = 'name'

Load the data from the CSV file (useful to avoid requesting all the data from Google if we already did it)

In [ ]:
unis = pd.read_csv('universities.csv', index_col='name')
unis.head()

Export the data to a CSV file.

In [ ]:
unis.to_csv('universities.csv')

We can see that we miss the location of 16 universities. We'll add the data manually. Note that some entries do not correspond to a specific place. For instance *weitere Institute* means other institutes and the first row corresponds to several libraries and museums. We'll leave these entries empty to exclude them from the analysis since we can't associate them to a specific canton.

In [ ]:
missing_loc = unis[unis.isnull().any(axis=1)]
print('There are {} entries without location data.'.format(len(missing_loc)))
missing_loc

In [ ]:
additional_data = {
    'Physikal.-Meteorolog. Observatorium Davos - PMOD': ('GR', 46.8133161, 9.8422335),
    'Swiss Center for Electronics and Microtech. - CSEM': ('NE', 46.997778, 6.9453443),
    'Fachhochschule Nordwestschweiz (ohne PH) - FHNW': ('SO', 47.348146, 7.9056693),
    'Swiss Institute of Bioinformatics - SIB': ('VD', 46.5205653, 6.5721676),
    'Forschungskommission SAGW': ('BE', 46.94727, 7.4344473),
    'Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP': ('TI', 46.02351, 8.9149293),
    'Schweizer Kompetenzzentrum Sozialwissensch. - FORS': ('VD', 46.5254827, 6.5784405),
    'Staatsunabh. Theologische Hochschule Basel - STHB': ('BS', 47.581238, 7.6479233)
}
new_df = pd.DataFrame.from_dict(additional_data, orient='index')
new_df.columns = ['canton', 'latitude', 'longitude']
unis.update(new_df) # Replace old data in the original dataframe

The 8 remaining entries could not be associated with a specific canton.

In [ ]:
missing_loc = unis[unis.isnull().any(axis=1)]
len(missing_loc)

We can also notice that some universities aren't located in Switzerland (e.g. the Istituto Svizzero di Roma). And thus we can ignore them since they aren't associated with any canton. We also remove the NaN values at the same time since we already said we couldn't associate them to any canton.

In [ ]:
swiss_cantons = ['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL',
                 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU']
unis = unis[unis.canton.isin(swiss_cantons)]

### Amount per canton
First, we create a new dataframe containing, for each project, the amount and the university. We need to convert the *amount* column to a numeric type since it's loaded from the CSV as a string.

In [ ]:
amounts = df[['University', 'Approved Amount']]
amounts = amounts.rename(columns={'University': 'university', 'Approved Amount': 'amount'})
amounts.index.name = 'project'
amounts.amount = pd.to_numeric(amounts.amount, errors='coerce')
amounts.head()

We can now merge the two dataframes to also get the associated canton for each project. Finally, we can group by canton and sum to get the total amount of funding per canton.

In [ ]:
amounts = amounts.merge(unis[['canton']], left_on='university', right_index=True, how='right')
amounts.head()

In [ ]:
amounts = amounts.groupby('canton').sum()
amounts = amounts.amount # The dataframe has only one column so we extract it to get a Pandas series
amounts.head()

The next cell makes sure all the Swiss cantons are in the `canton` series. We set the amount to 0 for the cantons that were missing.

In [ ]:
amounts = amounts.reindex(pd.Series(swiss_cantons)).fillna(0)

### Building the map
We build directly the map in Javascript using Leaflet insteaf of using Folium – which is a wrapper in Python around Leaflet. Our Javascript script is called `leaflet.js` and is located in the `js` folder.

We decided to use 6 classes to build our choropleth map since this we noticed that using more than 6 makes it harder to spot the differences between non-ajdacent cantons. The map is interactive: you can hover a canton to show the absolute amount of funding, you can also choose whether to display the universities or not, it is also possible to show a line indicating the Röstigraben location. Moreover, we converted the TopoJSON file containing the Swiss cantons geometry to the GeoJSON format since Leaflet doesn't support the TopoJSON format. We chose the color for our 6 classes using [ColorBrewer](http://colorbrewer2.org).

To chose split our data in 6 classes, we created the `linear_split(nb_classes)` function. It chooses the thresholds using the quantiles such that each class containts the same number of cantons.

In [223]:
def linear_split(nb_classes):
    linspace = np.linspace(amounts.min(), amounts.max(), nb_classes+1).round()
    # We don't want to include the min and max value in the scale so that it is compatible with our JS script
    return linspace[1:-1].tolist()

In [215]:
def quantile_split(nb_classes):
    # We ignore zero values so that cantons with no funding don't have an impact on the scale
    return [round(amounts[amounts > 0].quantile(i/nb_classes)) for i in range(1, nb_classes)]

To make our data available to our JS script, we export it to a *data.js* file which we load in the HTML page.

In [ ]:
def export_data(split_function):
    with open('js/data.js', 'w') as f:
        f.write('var universities = {}\n'.format(unis.reset_index().to_json(orient='records')))
        f.write('var cantons = {}\n'.format(amounts.to_json()))
        f.write('var scale = {}'.format(split_function(6)))

You can see the result below. Please note that the map won't show in the preview made by GitHub. You should clone the repository and either reevaluate the cells below or, after calling the `export_data()` function to choose which scale you want, open directly this [HTML page](leaflet.html).

### Quantiles scale

In [269]:
export_data(quantile_split)

In [270]:
%%HTML
<div style="position:relative;width:100%;height:0;padding-bottom:60%;">
<iframe src="leaflet.html" style="position:absolute;width:100%;height:100%;left:0;top:0;"></iframe></div>

### Linear scale

In [273]:
export_data(linear_split)

In [275]:
%%HTML
<div style="position:relative;width:100%;height:0;padding-bottom:60%;">
<iframe src="leaflet.html" style="position:absolute;width:100%;height:100%;left:0;top:0;"></iframe></div>

### Bonus: Röstigraben
BONUS: using the map you have just built, and the geographical information contained in it, could you give a rough estimate of the difference in research funding between the areas divided by the Röstigraben?

We know that the following cantons are french speaking:
- Vaud (VD), Neuchatel (NE), Jura (JU)

In some cantons, both languages are spoken:
- Valais (VS), Fribourg (FR), Berne (BE)

In Valais (VS), only one university is in the german speaking part. On top of that, it has only received an amount of CHF 12'000.-, which corresponds to $0.04$% of the money received by the canton and to les than $0.00001$% of the total amount. Thus, we count Valais as a french speaking canton.

Fribourg (FR) is a complicated case, because the university is exactly on the Röstigraben. We cannot say if the people who received the money in Fribourg universities speak french or german, and thus we discard it.

For Berne (BE), this is easy: all universities are in the german speaking country, we thus count it as a german speaking canton.

Tessin is a particular case, because people speak neither french nor german (based on the Röstigraben map from Wikipedia). We decided to not take it into account, because the Röstigraben is only a border splitting the french and german speaking parts in two based on language, it has nothing to do with the italian speaking part.

In [ ]:
french_cantons = ['VD', 'NE', 'JU', 'VS']
ignored_cantons = ['FR', 'TI']

Now, using the `amounts` table defined above, we can build the frame containing the amounts per canton for each region.

In [ ]:
# The french speaking cantons
data_french = amounts[amounts.index.isin(french_cantons)].to_frame()
data_french['language'] = 'French speaking'
data_french.set_index('language', inplace=True)

# The german speaking cantons
data_german = amounts[~amounts.index.isin(french_cantons+ignored_cantons)].to_frame()
data_german['language'] = 'German speaking'
data_german.set_index('language', inplace=True)

Finally, the data are concatenated, groupbed by index and summed.

In [ ]:
# Groupby index and sum the amounts per canton
rostigraben = pd.concat([data_french, data_german], axis=0)
grouped = rostigraben.groupby(rostigraben.index).sum()

# Plot
grouped.plot(y='amount', kind='pie', figsize=(6, 6), colormap='Pastel2', autopct='%1.1f%%')